# Interpretabilidad SimpleRNN

## Imports y funciones

In [ ]:
import sys
import numpy as np
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as mpatch

# Visualización 
def pinta_strings(estados, inputs, pinta_accept=False, probs=None, umbral=0.0):
    # estados: lista de arrays de estados en cada instante t.
    # inputs: caracteres guardados en formato numérico en cada instante t.
    # pinta_accept: Pinta las probs (solo valido para 3 simbolos).
    # probs: Si pinta_accept es true, probabilidades de cada simbolo (solo valido para 3 simbolos).
    
    inputs_print = ''.join([int_to_char[x] for x in inputs])

    # Accept or not
    if (pinta_accept):
        fig, ax = plt.subplots(figsize=(20,20))
        plt.ylabel("Probs")
        for i, simbolo in enumerate(inputs_print):
            rectangle = mpatch.Rectangle((i,0), 1, 1, color=(probs[i][0,0],probs[i][0,1],probs[i][0,2]))

            ax.add_artist(rectangle)
            rx, ry = rectangle.get_xy()
            cx = rx + rectangle.get_width()/2.0
            cy = ry + rectangle.get_height()/2.0

            ax.annotate(simbolo, (cx, cy), color='k', weight='bold',
                        fontsize=14, ha='center', va='center')

    ax.set_aspect('equal')
    ax.set_xlim((0, len(inputs)))
    plt.show()
            
    # Every neuron
    for ixNeurona in range(len(estados[0][0])):
        if np.mean(np.abs(estados[i][:, ixNeurona])) < umbral:
            continue
        fig, ax = plt.subplots(figsize=(20,20))
        plt.ylabel("N" + str(ixNeurona))
        for i, simbolo in enumerate(inputs_print):
            if estados[i][0, ixNeurona] < 0:
                rectangle = mpatch.Rectangle((i,0), 1, 1, color=(1.+estados[i][0,ixNeurona],
                                                                 1.,
                                                                 1.))
            else:
                rectangle = mpatch.Rectangle((i,0), 1, 1, color=(1.,
                                                                 1.-estados[i][0,ixNeurona],
                                                                 1.-estados[i][0,ixNeurona]))

            ax.add_artist(rectangle)
            rx, ry = rectangle.get_xy()
            cx = rx + rectangle.get_width()/2.0
            cy = ry + rectangle.get_height()/2.0

            ax.annotate(simbolo, (cx, cy), color='k', weight='bold',
                        fontsize=14, ha='center', va='center')

        ax.set_aspect('equal')
        ax.set_xlim((0, len(inputs)))
        plt.show()

# Generación del dataset
def generate_seq(numWords):
    cadena = ""
    alphabet = ["aa", "bbb", "aaaa", "bbbbb", "aaaaaa", "bbbbbbb"]
    return " ".join([alphabet[np.random.randint(6, size=1)[0]] for _ in range(numWords)])

## Generación de los datos

In [ ]:
raw_text = generate_seq(200000)

In [ ]:
# Mapeo de los carácteres a int
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# Resumen de los datos cargados
n_chars = len(raw_text)
n_vocab = len(chars)
print("Longitud total de carácteres: ", n_chars)
print("Número total de carácteres distintos: ", n_vocab)

In [ ]:
# Preparación del dataset de forma secuencial
seq_length = 20
data_x = []
data_y = []

for i in range(0, n_chars - seq_length, seq_length):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    data_x.append([char_to_int[char] for char in seq_in])
    data_y.append(char_to_int[seq_out])
    
n_patterns = len(data_x)
print("Número total de patrones: ", n_patterns)

## Entrenamiento de la red

In [ ]:
# Parámetros de la red
rnn_neurons = 3
epochs = 50
batch_size = 30
l1reg = 1e-4

In [ ]:
# One hot encoding de la entrada
x = np.reshape(tf.keras.utils.to_categorical(data_x), (n_patterns, seq_length, n_vocab))
y = tf.keras.utils.to_categorical(data_y)

# Definición de la capa de entrada
input = tf.keras.layers.Input(shape=(seq_length, n_vocab))

# Creación del RNN
if l1reg != 0:
    rnn = tf.keras.layers.SimpleRNN(rnn_neurons,name='rnn',
        activity_regularizer=tf.keras.regularizers.l1(l1reg))(input)
else:
    rnn = tf.keras.layers.SimpleRNN(rnn_neurons,name='rnn')(input)

# Añadida la capa de salida
dense = tf.keras.layers.Dense(n_vocab, activation='softmax')(rnn)

# Compilación del modelo
model = tf.keras.models.Model(inputs=input, outputs=[dense])
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Modelo intermedio para explorar el valor del peso recurrente
intermediate_layer_model = tf.keras.models.Model(inputs=model.input,
    outputs=model.get_layer('rnn').output)

# Entrenamiento de la red
model.fit(x, y, epochs=epochs, batch_size=batch_size)

## Predicción de la red

In [ ]:
# Se escoge una semilla inicial aleatoria
start = np.random.randint(0, len(data_x)-1)
pattern = data_x[start]
print("Semilla inicial:")
print(''.join([int_to_char[value] for value in pattern]))

# Generación de los carácteres
probs = []
estados = []
inputs = []

for i in range(50):
    x = np.reshape(tf.keras.utils.to_categorical(pattern,num_classes=n_vocab), 
        (1, len(pattern), n_vocab))
    
    # Predicción del siguiente carácter
    prediction = model.predict(x, verbose=0)
    probs.append(prediction)
    
    # Se guardan el valor de los pesos a interpretar
    estados.append(intermediate_layer_model.predict(x,verbose=0))
    
    # De forma aleatoria con la probabilidad dada por la red se elige el nuevo carácter
    index = np.random.choice(range(n_vocab), p=prediction.ravel())
    
    # Se guarda el valor para compararlo con la activación
    inputs.append(index)
    
    # Se añade de forma recursiva el carácter generado
    pattern.append(index)
    pattern = pattern[1:len(pattern)]


## Interpretabilidad

In [ ]:
# ROJO: ' '
# VERDE: 'a'
# AZUL: 'b'
pinta_strings(estados, inputs, pinta_accept=True, probs=probs, umbral=0)